In [ ]:
'''


用戶關注時，會將用戶id等資訊，寫成json，記錄在特定文字檔案內，並將特定菜單與用戶綁定

用戶發送文字消息，能簡單回傳文字消息，並簡單記錄在特定文字檔案內。

用戶發送貼圖消息，server端能回傳簡單貼圖消息

用戶發送圖片消息，server端能在特定資料夾收到圖片

用戶發送影片消息，server能在特定資料夾內搜集到影片

server能主動推送消息給 特定文字檔案內的用戶


'''

In [ ]:
'''

解釋Server基礎框架

三個組成大要件

Flask - HTTP Server
    使Line能將消息發至我方筆電

WebHookHandler
    Line發消息進來之後，處理Line消息，並進行分發的角色

LineBotApi
    使我方能夠與Line進行有效溝通


'''

In [1]:
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

import json

secretFileContentJson=json.load(open("../line_secret_key",'r'))


# 啟用Server
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 告知handler，若收到消息事件，且為文字消息的話，按下方處理
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # 回覆消息給Line，故使用line_bot_api此物件
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


In [2]:
'''

用戶發送文字消息，回應文字消息

'''

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # 回覆消息給Line，故使用line_bot_api此物件
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


In [3]:
'''

用戶關注時，會將用戶id等資訊，寫成json，記錄在特定文字檔案內，並將特定菜單與用戶綁定

參考資料
http://line-bot-sdk-python.readthedocs.io/en/latest/linebot.models.html#module-linebot.models.events

'''
from linebot.models.events import (
    JoinEvent,FollowEvent
)

import requests

@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出關注消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    # 將用戶資訊存在檔案內
    with open("../users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)

    # 發送文字回覆消息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="感謝關注，用戶菜單已綁定。")
    )
    


In [ ]:
'''

小結論

關於Chatbot

目前已經有一個業界基礎使用概念


用戶關注 -> 取得個資 -> 分析用戶 -> 配發不同菜單 -> 用戶發送文字消息 -> 系統回覆文字消息

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')


In [ ]:
'''

圖片/影片消息

用戶傳送圖片/影片消息，回應文字消息，並把照片下載到jupyter內的資料中

'''

@handler.add(MessageEvent, message=ImageMessage)
def download_image_to_server(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload')
    )
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('../images/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
    


In [ ]:
'''

作業  監控對話機器人

    將機器人拉入群內

      群內有用戶加入或離開時，紀錄其id
      群內有用戶發話時，紀錄其文字內容與圖片

'''

In [ ]:
'''

收到消息與寄發之消息 是不同格式與類別的


'''

In [14]:
'''

系統主動推送文字消息給用戶

    server能主動推送文字消息給 特定文字檔案內的用戶

'''

@app.route("/pushMessage", methods=['POST'])
def push_message():
    
    jsonObjectStrings = open("../users.txt",'r')
    jsonArray = []

    for jsonObjectString in jsonObjectStrings:
        jsonObject = json.loads(jsonObjectString)
        jsonArray.append(jsonObject)
    
    user_id_array = map(lambda user: user.user_id,jsonArray)
    
    line_bot_api.push_message(
        user_id_array,
        TextSendMessage("廣播消息")
    )


In [ ]:
'''

作業  系統主動推送圖片消息給用戶

    server能主動推送圖片消息給 特定文字檔案內的用戶

'''